Create Occupancy Grid Based on Free Space Estimation
Occupancy grids are used to represent a vehicle's surroundings as a discrete grid in vehicle coordinates and are used for path planning. Each cell in the occupancy grid has a value representing the probability of the occupancy of that cell. The estimated free space can be used to fill in values of occupancy grid.

The procedure to fill the occupancy grid using the free space estimate is as follows:

Define the dimensions of the occupancy grid in vehicle coordinates.

Generate a set of (X,Y) points for each grid cell. These points are in the vehicle's coordinate system.

Transform the points from the vehicle coordinate space (X,Y) into the bird's-eye-view image coordinate space (x,y) using the vehicleToImage transform.

Sample the free space confidence values at (x,y) locations using griddedInterpolant to interpolate free space confidence values that are not exactly at pixel centers in the image.

Fill the occupancy grid cell with the average free space confidence value for all (x,y) points that correspond to that grid cell.

For brevity, the procedure shown above is implemented in the supporting function, createOccupancyGridFromFreeSpaceEstimate, which is listed at the end of this example. Define the dimensions of the occupancy grid in terms of the bird's-eye-view configuration and create the occupancy grid by calling createOccupancyGridFromFreeSpaceEstimate.

In [ ]:
#Define dimensions and resolution of the occupancy grid.
gridX = distAheadOfSensor
gridY = 2 * spaceToOneSide
cellSize = 0.25 # in meters to match units used by CamVid sensor

# Create the occupancy grid from the free space estimate.
occupancyGrid = createOccupancyGridFromFreeSpaceEstimate(
    freeSpaceBEV, birdsEyeConfig, gridX, gridY, cellSize)

Create Vehicle Costmap Using the Occupancy Grid
The vehicleCostmap provides functionality to check if locations, in vehicle or world coordinates, are occupied or free. This check is required for any path-planning or decision-making algorithm. Create the vehicleCostmap using the generated occupancyGrid.

In [ ]:
def createOccupancyGridFromFreeSpaceEstimate(
    freeSpaceBEV,birdsEyeConfig,gridX,gridY,cellSize):
    # Return an occupancy grid that contains the occupancy probability over
    # a uniform 2-D grid.

    # Number of cells in occupancy grid.
    numCellsX = ceil(gridX / cellSize)
    numCellsY = ceil(gridY / cellSize)

    # Generate a set of (X,Y) points for each grid cell. These points are in
    # the vehicle's coordinate system. Start by defining the edges of each grid
    # cell.

    # Define the edges of each grid cell in vehicle coordinates.
    XEdges = linspace(0,gridX,numCellsX)
    YEdges = linspace(-gridY/2,gridY/2,numCellsY)

    # Next, specify the number of sample points to generate along each
    # dimension within a grid cell. Use these to compute the step size in the
    # X and Y direction. The step size will be used to shift the edge values of
    # each grid to produce points that cover the entire area of a grid cell at
    # the desired resolution.

    # Sample 20 points from each grid cell. Sampling more points may produce
    # smoother estimates at the cost of additional computation.
    numSamplePoints = 20;

    # Step size needed to sample number of desired points.
    XStep = (XEdges(2)-XEdges(1)) / (numSamplePoints-1)
    YStep = (YEdges(2)-YEdges(1)) / (numSamplePoints-1)

    # Finally, slide the set of points across both dimensions of the grid
    # cells. Sample the occupancy probability along the way using
    # griddedInterpolant.

    # Create griddedInterpolant for sampling occupancy probability. Use 1
    # minus the free space confidence to represent the probability of occupancy.
    occupancyProb = 1 - freeSpaceBEV
    sz = size(occupancyProb)
    [y,x] = ndgrid(1:sz(1),1:sz(2))
    F = griddedInterpolant(y,x,occupancyProb)

    # Initialize the occupancy grid to zero.
    occupancyGrid = zeros(numCellsY*numCellsX,1)

    # Slide the set of points XEdges and YEdges across both dimensions of the
    # grid cell. 
    for j = 1:numSamplePoints
        
        # Increment sample points in the X-direction
        X = XEdges + (j-1)*XStep;
    
        for i = 1:numSamplePoints
            
            # Increment sample points in the Y-direction
            Y = YEdges + (i-1)*YStep;
            
            # Generate a grid of sample points in bird's-eye-view vehicle coordinates
            [XGrid,YGrid] = meshgrid(X,Y);
            
            # Transform grid of sample points to image coordinates
            xy = vehicleToImage(birdsEyeConfig,[XGrid(:) YGrid(:)]);
            
            # Clip sample points to lie within image boundaries
            xy = max(xy,1);
            xq = min(xy(:,1),sz(2));        
            yq = min(xy(:,2),sz(1));
            
            # Sample occupancy probabilities using griddedInterpolant and keep
            # a running sum.
            occupancyGrid = occupancyGrid + F(yq,xq);  


    # Determine mean occupancy probability.
    occupancyGrid = occupancyGrid / numSamplePoints^2;
    occupancyGrid = reshape(occupancyGrid,numCellsY,numCellsX);

In [ ]:
# Create the costmap.
costmap = vehicleCostmap(flipud(occupancyGrid),
    'CellSize',cellSize,
    'MapLocation',[0,-spaceToOneSide])
costmap.CollisionChecker.InflationRadius = 0;
